In [2]:
def process_aspect_and_sentiment(file_path, output_file_path):
    processed_data = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
        # Xử lý từng nhóm 3 dòng: câu, khía cạnh, sentiment
        for i in range(0, len(lines), 3):
            sentence = lines[i].strip()
            aspect = lines[i+1].strip()
            sentiment = lines[i+2].strip()
            
            # Thay thế $T$ bằng khía cạnh và thêm token đặc biệt
            marked_sentence = sentence.replace('$T$', f"<aspect> {aspect} </aspect>")
            
            # Kết hợp câu đã đánh dấu với khía cạnh và sentiment
            processed_data.append(f"{marked_sentence}\t{aspect}\t{sentiment}")
    
    # Lưu dữ liệu đã xử lý vào file mới
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for line in processed_data:
            output_file.write(line + '\n')

# Gọi hàm để xử lý file train và test
process_aspect_and_sentiment('train.raw', 'final_processed_train.tsv')
process_aspect_and_sentiment('test.raw', 'final_processed_test.tsv')


In [5]:
import re

# Hàm để loại bỏ các ký tự không phù hợp mà vẫn giữ token đặc biệt như <aspect> và </aspect>
def remove_unwanted_characters(sentence):
    # Loại bỏ ký tự đặc biệt như *** và hashtag
    sentence = re.sub(r"\*+", "", sentence)  # Loại bỏ ký tự *
    sentence = re.sub(r"#\S+", "", sentence)  # Loại bỏ hashtag
    sentence = re.sub(r"http\S+", "", sentence)  # Loại bỏ URL
    # Loại bỏ các ký tự không phải là chữ cái, số, dấu câu, hoặc các token đặc biệt <aspect> </aspect>
    sentence = re.sub(r"[^\w\s,.!?:;<>/]", "", sentence)
    return sentence

# Hàm xử lý toàn bộ dataset chỉ loại bỏ ký tự không phù hợp
def process_dataset_remove_unwanted(file_path, output_file_path):
    cleaned_data = []
    
    # Đọc dữ liệu từ file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    for line in lines:
        parts = line.strip().split("\t")
        if len(parts) == 3:
            sentence, aspect, sentiment = parts
            # Loại bỏ ký tự không phù hợp trong câu văn bản và khía cạnh
            cleaned_sentence = remove_unwanted_characters(sentence)
            cleaned_aspect = remove_unwanted_characters(aspect)
            cleaned_data.append(f"{cleaned_sentence}\t{cleaned_aspect}\t{sentiment}")
    
    # Lưu dữ liệu đã làm sạch vào file mới
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for line in cleaned_data:
            output_file.write(line + '\n')

# Gọi hàm để xử lý và lưu file
process_dataset_remove_unwanted('final_processed_train.tsv', 'cleaned_final_processed_train.tsv')
process_dataset_remove_unwanted('final_processed_test.tsv', 'cleaned_final_processed_test.tsv')


In [1]:
import re

# Hàm để loại bỏ dấu chấm trong khía cạnh (aspect)
def remove_period_from_aspect(aspect):
    # Loại bỏ dấu chấm ở đầu, cuối, và giữa khía cạnh
    return aspect.replace('.', '')

# Hàm xử lý toàn bộ dataset để loại bỏ dấu chấm trong khía cạnh
def process_dataset_remove_period(file_path, output_file_path):
    cleaned_data = []
    
    # Đọc dữ liệu từ file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Xử lý từng dòng dữ liệu
    for line in lines:
        parts = line.strip().split("\t")
        if len(parts) == 3:
            sentence, aspect, sentiment = parts
            # Loại bỏ dấu chấm trong khía cạnh
            cleaned_aspect = remove_period_from_aspect(aspect)
            cleaned_data.append(f"{sentence}\t{cleaned_aspect}\t{sentiment}")
    
    # Lưu dữ liệu đã làm sạch vào file mới
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for line in cleaned_data:
            output_file.write(line + '\n')

# Gọi hàm để xử lý file train và test
process_dataset_remove_period('cleaned_final_processed_train.tsv', 'cleaned_final_processed_train_no_period.tsv')
process_dataset_remove_period('cleaned_final_processed_test.tsv', 'cleaned_final_processed_test_no_period.tsv')

print("Xử lý hoàn tất, file đã được lưu.")


Xử lý hoàn tất, file đã được lưu.


In [1]:
import pandas as pd

# Đọc file train và test
train_data = pd.read_csv('train_final.tsv', sep='\t', header=None, names=['sentence', 'aspect', 'sentiment'])
test_data = pd.read_csv('test_final.tsv', sep='\t', header=None, names=['sentence', 'aspect', 'sentiment'])

# Thay đổi các giá trị sentiment:
# - `-1` (tiêu cực) sẽ thành `0`
# - `0` (trung lập) sẽ thành `1`
# - `1` (tích cực) sẽ thành `2`
train_data['sentiment'] = train_data['sentiment'].map({-1: 0, 0: 1, 1: 2})
test_data['sentiment'] = test_data['sentiment'].map({-1: 0, 0: 1, 1: 2})

# Lưu lại file đã chỉnh sửa
train_data.to_csv('train_final_fixed.tsv', sep='\t', index=False, header=False)
test_data.to_csv('test_final_fixed.tsv', sep='\t', index=False, header=False)


In [2]:
import pandas as pd

# Đọc file train và test
train_data = pd.read_csv('train_final_fixed.tsv', sep='\t', header=None, names=['sentence', 'aspect', 'sentiment'])
test_data = pd.read_csv('test_final_fixed.tsv', sep='\t', header=None, names=['sentence', 'aspect', 'sentiment'])

# Kiểm tra xem có giá trị NaN nào không
print("NaN trong train_data:")
print(train_data.isna().sum())

print("NaN trong test_data:")
print(test_data.isna().sum())

# Loại bỏ các hàng chứa NaN nếu có
train_data = train_data.dropna()
test_data = test_data.dropna()

# Lưu lại dữ liệu đã xử lý
train_data.to_csv('train_final_cleaned.tsv', sep='\t', index=False, header=False)
test_data.to_csv('test_final_cleaned.tsv', sep='\t', index=False, header=False)


NaN trong train_data:
sentence     0
aspect       2
sentiment    0
dtype: int64
NaN trong test_data:
sentence     0
aspect       1
sentiment    0
dtype: int64


In [3]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('train_final_cleaned.tsv', sep='\t', header=None, names=['sentence', 'aspect', 'sentiment'])
test_data = pd.read_csv('test_final_cleaned.tsv', sep='\t', header=None, names=['sentence', 'aspect', 'sentiment'])

# Check for NaN values in the train and test datasets
train_nan = train_data.isna().sum()
test_nan = test_data.isna().sum()

print("Train dataset NaN values:\n", train_nan)
print("Test dataset NaN values:\n", test_nan)

Train dataset NaN values:
 sentence     0
aspect       0
sentiment    0
dtype: int64
Test dataset NaN values:
 sentence     0
aspect       0
sentiment    0
dtype: int64


In [76]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Đường dẫn tới file từ điển khía cạnh
ASPECT_DICT_PATH = 'aspect_dict.txt'  # Thay đổi đường dẫn nếu cần thiết

# Hàm đọc từ điển khía cạnh từ file
def load_aspect_dict(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        aspect_keywords = f.read().splitlines()
    return set(aspect_keywords)  # Chuyển thành set để dễ kiểm tra từ khóa

# Load từ điển khía cạnh
aspect_dict = load_aspect_dict(ASPECT_DICT_PATH)

# Load PhoBERT tokenizer và model đã huấn luyện
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("D:/ABSAPhoBert/train_model")  # Đường dẫn thư mục lưu mô hình sau khi huấn luyện
model.eval()  # Đặt model vào chế độ evaluation (đánh giá)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Hàm để mã hóa và dự đoán sentiment cho một khía cạnh
def predict_sentiment(sentence, aspect):
    modified_sentence = f"Về khía cạnh {aspect}, {sentence}"
    inputs = tokenizer.encode_plus(modified_sentence, aspect, add_special_tokens=True, truncation=True, max_length=256, padding="max_length", return_tensors="pt", truncation_strategy='only_first')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    
    return predicted_class

# Sử dụng từ điển khía cạnh để xác định khía cạnh trong câu
def identify_aspects(sentence, aspect_dict):
    identified_aspects = []
    for keyword in aspect_dict:
        if keyword in sentence:
            identified_aspects.append(keyword)
    return identified_aspects

# Hàm để dự đoán sentiment cho các khía cạnh được nhận diện
def predict_sentiment_for_identified_aspects(sentence, aspect_dict):
    aspects = identify_aspects(sentence, aspect_dict)
    results = {}
    for aspect in aspects:
        predicted_class = predict_sentiment(sentence, aspect)
        
        if predicted_class == 0:
            sentiment = "negative"
        elif predicted_class == 1:
            sentiment = "neutral"
        else:
            sentiment = "positive"
        
        results[aspect] = sentiment
    
    return results

# Ví dụ câu cần phân tích
sentence = "Sản phẩm tốt nhưng giao hàng chậm"

# Dự đoán sentiment cho các khía cạnh được nhận diện
predicted_sentiments = predict_sentiment_for_identified_aspects(sentence, aspect_dict)

# Hiển thị kết quả
for aspect, sentiment in predicted_sentiments.items():
    print(f"Khía cạnh: {aspect} - Sentiment: {sentiment}")


Khía cạnh: Sản phẩm - Sentiment: positive
Khía cạnh: hàng - Sentiment: negative
Khía cạnh: gia - Sentiment: negative
Khía cạnh: giao - Sentiment: negative
Khía cạnh: giao hàng - Sentiment: negative


In [3]:
import torch
print(torch.cuda.is_available())  # Nếu trả về True, bạn đang dùng GPU

True


In [5]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\huynh\AppData\Roaming\nltk_data...


True

In [16]:
!python infer_example.py --sentence "Sản phẩm chính hãng và khá là ưa dùng nhưng shipper giao hàng rất chậm"

Extracted aspect: sản phẩm
Combined phrases: --sentence sản phẩm chính hãng và khá là ưa dùng
Prediction for aspect ('sản phẩm'): Positive
Extracted aspect: giao hàng
Combined phrases: shipper giao hàng rất chậm
Prediction for aspect ('giao hàng'): Negative


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

In [2]:
!python infer_example.py --sentence "Shipper thân thiện"

Extracted aspect: shipper
Combined phrases: shipper thân thiện
Prediction for aspect ('shipper'): Positive


c:\Users\huynh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. I

In [17]:
!python infer_example.py --sentence "Shipper khá là khó chịu và gây khó dễ cho người dùng"

Extracted aspect: shipper
Combined phrases: --sentence shipper khá là khó chịu và gây khó dễ cho người dùng
Prediction for aspect ('shipper'): Negative


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

In [18]:
!python infer_example.py --sentence "Hàng tốt"

Extracted aspect: hàng
Combined phrases: --sentence hàng tốt
Prediction for aspect ('hàng'): Positive


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

In [19]:
!python infer_example.py --sentence "giao hàng chậm"

Extracted aspect: giao hàng
Combined phrases: --sentence giao hàng chậm
Prediction for aspect ('giao hàng'): Negative


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

In [20]:
!python infer_example.py --sentence "dùng tốt"

Extracted aspect: dùng
Combined phrases: --sentence dùng tốt
Prediction for aspect ('dùng'): Positive


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

In [15]:
!python infer_example.py --sentence "Sử dụng rất ổn định"

Extracted aspect: sử dụng
Combined phrases: sử dụng rất ổn định
Prediction for aspect ('sử dụng'): Positive


In [19]:
!python infer_example.py --sentence "Sản phẩm sử dụng rất ổn định"

Extracted aspect: sản phẩm
Combined phrases: sản phẩm sử dụng rất ổn định
Prediction for aspect ('sản phẩm'): Positive


In [2]:
!python infer_example.py --sentence "Sản phẩm dùng khá ổn nhưng thời gian giao hàng khá lâu"

Extracted aspect: sản phẩm

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 


Combined phrases: --sentence sản phẩm dùng khá ổn
Prediction for aspect ('sản phẩm'): Positive
Extracted aspect: thời gian giao hàng
Combined phrases: thời gian giao hàng khá lâu
Prediction for aspect ('thời gian giao hàng'): Negative


In [1]:
!python infer_example.py --sentence "Một số trường hợp không được đồng kiểm, hộp điện thoại đã bị khui, chăm sóc khách hàng chưa tốt. Một số máy bị đơ và nóng khi sử dụng."

Extracted aspect: hộp điện thoại
Combined phrases: hộp điện thoại đã bị khui
Prediction for aspect ('hộp điện thoại'): Negative
Extracted aspect: chăm sóc khách hàng
Combined phrases: chăm sóc khách hàng chưa tốt
Prediction for aspect ('chăm sóc khách hàng'): Negative
Extracted aspect: sử dụng
Combined phrases: một số máy bị đơ và nóng khi sử dụng
Prediction for aspect ('sử dụng'): Negative


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\git hub desktop\ABSA-PhoBert-and-Bayes\infer_example.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode 

In [9]:
!python infer_example.py --sentence "hộp điện thoại đã bị khui, chăm sóc khách hàng chưa tốt. Một số máy bị đơ và nóng khi sử dụng. Sản phẩm còn new seal. Mình đã từng dùng cả Xiao Mi và Samsung thì thấy Samsung phần mềm ổn định, dễ dùng hơn, ít lỗi vặt hơn, phần cứng bền bỉ hơn và không bị nóng máy."

Extracted aspect: hộp điện thoại
Combined phrases: --sentence hộp điện thoại đã bị khui
Prediction for aspect ('hộp điện thoại'): Negative
Extracted aspect: chăm sóc khách hàng
Combined phrases: chăm sóc khách hàng chưa tốt
Prediction for aspect ('chăm sóc khách hàng'): Negative
Extracted aspect: sử dụng
Combined phrases: một số máy bị đơ và nóng khi sử dụng
Prediction for aspect ('sử dụng'): Negative
Extracted aspect: sản phẩm
Combined phrases: sản phẩm còn new seal
Prediction for aspect ('sản phẩm'): Neutral
Extracted aspect: phần mềm
Combined phrases: mình đã từng dùng cả xiao mi và samsung thì thấy samsung phần mềm ổn định
Prediction for aspect ('phần mềm'): Positive
Extracted aspect: dùng
Combined phrases: dễ dùng hơn
Prediction for aspect ('dùng'): Positive
Extracted aspect: phần cứng
Combined phrases: phần cứng bền bỉ hơn và không bị nóng máy
Prediction for aspect ('phần cứng'): Positive


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!python infer_example.py --sentence "Sản phẩm không quá mắc tiền"

Extracted aspect: sản phẩm
Combined phrases: --sentence sản phẩm không quá mắc tiền
Prediction for aspect ('sản phẩm'): Positive


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
!python infer_example.py --sentence "Sản phẩm bình thường"

Extracted aspect: sản phẩm
Combined phrases: --sentence sản phẩm bình thường
Prediction for aspect ('sản phẩm'): Neutral


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
!python infer_example.py --sentence "Sản phẩm chính hãng"

Extracted aspect: sản phẩm
Combined phrases: --sentence sản phẩm chính hãng
Prediction for aspect ('sản phẩm'): Neutral


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!python infer_example.py --sentence "Sản phẩm tệ song theo đó là sử dụng cũng rất là chán"

Extracted aspect: sản phẩm
Combined phrases: sản phẩm tệ
Prediction for aspect ('sản phẩm'): Negative
Extracted aspect: sử dụng
Combined phrases: theo đó là sử dụng cũng rất là chán
Prediction for aspect ('sử dụng'): Negative


In [6]:
!python infer_example.py --sentence "Sản phẩm tốt, chính hãng, dễ sử dụng"

Extracted aspect: sản phẩm
Combined phrases: sản phẩm tốt
Prediction for aspect ('sản phẩm'): Positive
Extracted aspect: sử dụng
Combined phrases: dễ sử dụng
Prediction for aspect ('sử dụng'): Positive


In [7]:
!python infer_example.py --sentence "Sản phẩm ở mức khá"

Extracted aspect: sản phẩm
Combined phrases: sản phẩm ở mức khá
Prediction for aspect ('sản phẩm'): Neutral


In [8]:
!python infer_example.py --sentence "Sản phẩm tạm được"

Extracted aspect: sản phẩm
Combined phrases: sản phẩm tạm được
Prediction for aspect ('sản phẩm'): Neutral


In [9]:
!python infer_example.py --sentence "Giao hàng nhanh nhưng sản phẩm không tốt"

Extracted aspect: giao hàng
Combined phrases: giao hàng nhanh
Prediction for aspect ('giao hàng'): Positive
Extracted aspect: sản phẩm
Combined phrases: sản phẩm không tốt
Prediction for aspect ('sản phẩm'): Negative


In [8]:
!python infer_example.py --sentence "Tôi cảm thấy khá là may mắn khi mua được sản phẩm này, đối với việc phải trải qua những sản phẩm dễ bị hư trong quá khứ tôi đã được trải nghiệm sản phẩm mới và khá chất lượng"

Extracted aspect: sản phẩm
Combined phrases: --sentence tôi cảm thấy khá là may mắn khi mua được sản phẩm này
Prediction for aspect ('sản phẩm'): Positive
Extracted aspect: trải nghiệm
Combined phrases: đối với việc phải trải qua những sản phẩm dễ bị hư trong quá khứ tôi đã được trải nghiệm sản phẩm mới và khá chất lượng
Prediction for aspect ('trải nghiệm'): Positive


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
